## Scraping Absentee and Early Voting results from state Board of Elections website

In [1]:
import pandas as pd
import csv
import requests
from bs4 import BeautifulSoup

In [2]:
counties = pd.read_csv('input/countylist.csv', dtype = {'id':str})
counties['id'] = counties['id'].str.zfill(2)

In [3]:
counties

,id,county
0,01,Allegany
1,02,Anne Arundel
2,03,Baltimore City
3,04,Baltimore
4,05,Calvert
5,06,Caroline
6,07,Carroll
7,08,Cecil
8,09,Charles
9,10,Dorchester


In [3]:
for i in counties['id']:
    r = requests.get('https://elections.maryland.gov/elections/2018/results/general/gen_results_2018_2_by_county_' + 
                 format(i) + '-1.html')
    
    soup = BeautifulSoup(r.content,'html.parser')
    
    tables = soup.find_all(lambda tag: tag.name == 'table')
    candidates = soup.select('td[headers="CandidateName003-"]')
    early_votes = soup.select('td[headers="EarlyVotes003-"]')
    election_day_votes = soup.select('td[headers="ElectionNightVotes003-"]')
    absentee_prov_votes = soup.select('td[headers="AbsenteeAndProvisionalVotes003-"]')
    total_votes = soup.select('td[headers="TotalVotes003-"]')
    percentages = soup.select('td[headers="Percentage003-"]')
    
    cands = []

    for candidate in candidates:
       # print(candidate.text.strip())
        cands.append(candidate.text.strip())
    
    early = []
    for early_vote in early_votes:
       # print(early_vote.text.strip())
        early.append(early_vote.text.strip())
    
    electday = []
    for election_day_vote in election_day_votes:
       # print(election_day_vote.text.strip())
        electday.append(election_day_vote.text.strip())
    
    absentee = []
    for absentee_prov_vote in absentee_prov_votes:
       # print(absentee_prov_vote.text.strip())
        absentee.append(absentee_prov_vote.text.strip())
    
    total = []
    for total_vote in total_votes:
       # print(total_vote.text.strip())
        total.append(total_vote.text.strip())
         
    perc = []
    for percentage in percentages:
       # print(percentage.text.strip())
        perc.append(percentage.text.strip())
        
        
    table_list = pd.DataFrame(
    {'candidate': cands,
     'early_vote': early,
     'electday_vote': electday,
     'absentee_vote':absentee,
     'total_vote':total,
     'perc':perc,
     'id':i
    })
    
    table_list = pd.merge(table_list, counties[['id', 'county']], on = 'id')
    
    table_list.to_csv('output/govresults' + i + '.csv', index = False)